Este libro muestra una mejor forma de hacerlo

In [1]:
import pandas as pd
#pd.options.display.max_rows = None

sol_cred = pd.read_csv('../files/input/solicitudes_de_credito.csv', sep=';', index_col=0)
sol_cred.dropna(inplace=True)

In [2]:
def limpieza_strings(dataset, campo):

    import re
    dataset.copy()

    dataset[campo] = dataset[campo].str.lower()
    dataset[campo] = dataset[campo].str.replace('_' , ' ')
    dataset[campo] = dataset[campo].str.replace('-' , ' ')
    dataset[campo] = dataset[campo].str.replace('.' , '')
    dataset[campo] = dataset[campo].str.replace('soli diaria' , 'solidaria')
    dataset[campo] = dataset[campo].apply(lambda x: re.sub(r"\s+", " ", str(x).replace("\t", " ")) if pd.notnull(x) else x)
    dataset[campo] = dataset[campo].str.strip()

    return dataset

In [3]:
def tokenizar_frase(frase):

    try:
        import nltk
        nltk.data.find('tokenizers/punkt')
        nltk.data.find('corpora/stopwords')
    except (ImportError, LookupError):
        !pip install nltk
        nltk.download('punkt')
        nltk.download('stopwords')
    
    import nltk
    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords

    spanish_stopwords = set(stopwords.words('spanish'))

    tokens = word_tokenize(frase)
    frase_limpia = " ".join([word for word in tokens if word not in spanish_stopwords])

    return frase_limpia


In [4]:
def limpieza_fechas(dataset, campo):

    dataset.copy()

    dataset['fecha_homologada'] = pd.to_datetime(dataset[campo], dayfirst=True, errors='coerce')
    no_identificadas = dataset['fecha_homologada'].isnull()
    dataset.loc[no_identificadas, 'fecha_homologada'] = pd.to_datetime(dataset.loc[no_identificadas, campo], format="%Y/%m/%d", errors='coerce')

    dataset.drop(columns=campo, inplace=True)
    dataset.rename(columns={'fecha_homologada':'fecha_de_beneficio'}, inplace=True)

    return dataset

In [5]:
def limpieza_numeros(valor):

    import re

    valor = valor.replace('$' , '')
    valor = valor.replace(',' , '')
    valor = valor.replace('.00' , '')
    valor = re.sub(r"\s+", "", valor)
    valor = valor.strip()

    return int(float(valor))

In [6]:
sol_cred = limpieza_strings(dataset=sol_cred, campo='sexo')
sol_cred = limpieza_strings(dataset=sol_cred, campo='tipo_de_emprendimiento')
sol_cred = limpieza_strings(dataset=sol_cred, campo='idea_negocio')
sol_cred = limpieza_strings(dataset=sol_cred, campo='barrio')
sol_cred = limpieza_strings(dataset=sol_cred, campo='línea_credito')
sol_cred['idea_negocio'] = sol_cred['idea_negocio'].apply(tokenizar_frase)
sol_cred['barrio'] = sol_cred['barrio'].apply(tokenizar_frase)
sol_cred = limpieza_fechas(dataset=sol_cred, campo='fecha_de_beneficio')
sol_cred['monto_del_credito'] = sol_cred['monto_del_credito'].apply(lambda x: limpieza_numeros(x) if isinstance(x, str) else x)

In [7]:
sol_cred.drop_duplicates(inplace=True)

In [8]:
import os

output_directory = os.path.join('files', 'output')
output_file = os.path.join(output_directory, 'solicitudes_de_credito.csv')

os.makedirs(output_directory, exist_ok=True)
sol_cred.to_csv(output_file, index=False, sep=';')